In [0]:
# =============================================================
# JOB 1 — INPE RAW DOWNLOAD
# =============================================================

import os, re, time, requests
from datetime import datetime

try:
    dbutils.widgets.text("DATA_ALVO", datetime.now().strftime("%Y%m%d"))
    DATA_ALVO = dbutils.widgets.get("DATA_ALVO")
except Exception:
    DATA_ALVO = datetime.now().strftime("%Y%m%d")

print(f"📅 Baixando dados INPE para o dia: {DATA_ALVO}")

BASE_URL = "https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/"
HEADERS = {"User-Agent": "inpe-focos-raw/3.0"}
MIN_TAMANHO = 1024

LANDING_PATH = f"/Volumes/datamasters/raw/raw_inpe/stream/inpe_in/{DATA_ALVO}"
dbutils.fs.mkdirs(f"dbfs:{LANDING_PATH}")

def listar_csvs():
    r = requests.get(BASE_URL, headers=HEADERS, timeout=30)
    r.raise_for_status()
    arquivos = re.findall(r'"([^"]+\.csv)"', r.text)
    return [a for a in arquivos if DATA_ALVO in a]

baixados = 0
for nome in listar_csvs():
    url = BASE_URL + nome
    r = requests.get(url, headers=HEADERS, timeout=60)
    if r.status_code == 200 and len(r.content) > MIN_TAMANHO:
        tmp = f"/tmp/{nome}"
        with open(tmp, "wb") as f:
            f.write(r.content)
        dbutils.fs.cp(f"file:{tmp}", f"dbfs:{LANDING_PATH}/{nome}")
        os.remove(tmp)
        print(f"[✔] {nome}")
        baixados += 1
    else:
        print(f"[✖] {nome}: arquivo inválido")

print(f"✅ Total de arquivos baixados: {baixados}")
dbutils.notebook.exit(f"RAW_OK_{DATA_ALVO}")